<a href="https://colab.research.google.com/github/Zagusan/Wikibot-3000/blob/main/Wikibot_3000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para crear a **Wikibot-3000** ocupamos hacer las siguientes cosas:

*   Preparar nuestro programa para que conozca todos los comandos requeridos
*   Obtener un archivo con los datos con los que nuestro modelo de lenguaje aprenderá
*   Entrenar, evaluar y probar el modelo

Todo esto se detallará a través de este experimento.


# Terminología

### TPU

Tensor Processing Unit por sus siglas en inglés. Es un componente especializado que se usa en las computadoras para acelerar los trabajos relacionados a la inteligencia artifical. Los servidores de Google trabajan con `TPUs`, sin embargo, dependiendo de tu dispositivo elegido, también se pueden llamar "Tensor Cores", "Neural Engine" o "Unidades XMX".

### Transformer

Se usa para referirse a una arquitectura de inteligencia artificial que funciona aprendiendo de grandes cantidades de texto y que al recibir una entrada, predicen cual será la respuesta más popular o acertada. Esta es responsable de permitir que servicios como ChatGPT y Stable Diffusion funcionen.

### Token

Serie de números que representan a una letra. Hay ciertas letras que tienen más tokens que otras

### Tokenizar

Proceso mediante el cual se traduce algo de texto a números para que una computadora pueda entenderlo

# Preparaciones

Primero verifiquemos la conexión con los servidores de Google y aseguremonos de que haya una `TPU` disponible.

In [1]:
from typing import Text
import tensorflow as tf

print("Versión de TensorFlow " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Ejecutando en TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('Error: No hay conexión con una TPU')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Versión de TensorFlow 2.12.0
Ejecutando en TPU  ['10.15.126.122:8470']


Luego, es necesario instalar los módulos a utilizar.

In [2]:
! pip install datasets transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.2 MB/s eta 0:00:00


In [3]:
! apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.


También es necesario comprobar que la versión de `transformers` sea mayor a 4.11

In [4]:
import transformers
print(transformers.__version__)

4.33.2


In [5]:
import datasets
print(datasets.__version__)

2.14.5


In [6]:
import accelerate
print(accelerate.__version__)

0.23.0


Por último, es necesario iniciar sesión en [Hugging Face](https://huggingface.co) para poder compartir nuestra IA con facilidad al finalizar.

In [7]:
from huggingface_hub import notebook_login

notebook_login()

# Datos

Para obtener nuestros datos, lo primero escoger nuestro `dataset`, y además de eso la variación de este, llamada `split`. Se pueden inspeccionar con el siguiente código:

In [ ]:
from datasets import get_dataset_split_names
from datasets import load_dataset_builder

print('Splits del dataset:')
print(get_dataset_split_names('graelo/wikipedia', '20230901.es'))
print('\n')

ds_builder = load_dataset_builder('graelo/wikipedia', '20230901.es')

print('Descripción:\n' + ds_builder.info.description + '\n')

print('Características Disponibles: \n')
print(ds_builder.info.features)
print('\n')

Ahora, hay que obtener una base de datos. Dependiendo de tu objetivo, puedes modificar `load_dataset()` para que obtenga distintas bases de datos.

In [8]:
from datasets import load_dataset

data = load_dataset('graelo/wikipedia', '20230901.es', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1826609 [00:00<?, ? examples/s]

Esto es lo que nuestro `dataset` contiene por dentro:

In [2]:
print(data[1])

{'id': '38', 'url': 'https://es.wikipedia.org/wiki/Asaph%20Hall', 'title': 'Asaph Hall', 'text': 'Asaph Hall (Goshen, Connecticut; 15 de octubre de 1829-Annapolis, Maryland; 22 de noviembre de 1907) fue un astrónomo estadounidense. Descubridor en 1877 de los dos satélites de Marte —Fobos y Deimos—, determinó las órbitas de numerosos satélites, el movimiento de una serie de estrellas binarias, la rotación de Saturno y la masa de Marte y de otros planetas.\n\nBiografía \nCon una escasa formación académica, ya que a la muerte de su padre tuvo que trabajar como carpintero para ayudar a su madre, sus ansias de conocimiento y de estudiar el cielo le llevaron a prepararse por su propia cuenta. En 1857 entró a trabajar como ayudante del astrónomo William Cranch Bond en el Observatorio de Harvard (Harvard College Observatory): el sueldo que tenía era de doce dólares al mes.\n\nEn 1856 contrajo matrimonio con Angeline Stickney (1830 - 1892), quien siempre le apoyó en su trabajo.\n\nDespués de ef

Como se pudo observar en la celda anterior, el `dataset` tiene:
* IDs
* URLs
* Títulos
* Texto

Solo necesitamos el texto, así que quitaremos el resto.

In [9]:
data = data.map(batched=True, batch_size=128, num_proc=4 ,remove_columns=['id', 'url', 'title'])

Map (num_proc=4):   0%|          | 0/1826609 [00:00<?, ? examples/s]

El resultado es que ahora solo tenemos texto:

In [14]:
print(data['train'][1])

{'text': 'Barrio es un concejo del municipio de Valdegovía, en la provincia de Álava, País Vasco (España).\n\nLocalización \nBarrio se encuentra a 5 de kilómetros de Espejo, la localidad más importante del municipio y única vía de entrada al pueblo por carretera.\n\nGeografía \nEs un pueblo de montaña enclavado en un cerrado valle a los pies del monte Bachicabo que vive principalmente de la ganadería.\n\nLos bosques de la localidad y el entorno son ricos y bien conservados, con frondosos robledales, encinales y hayedos.\n\nDespoblado  \nForma parte del concejo el despoblado de:\n Berbea.\nForma parte del concejo una fracción del despoblado de:\n Medropio.\n\nDemografía\n\nMonumentos \n Iglesia de Santa María.\n Ermita de la Virgen de Mellera\n\nReferencias\n\nEnlaces externos \n\nConcejos de Valdegovía'}


Como la base de datos que escogimos solo tiene un `split`, hay que dividirla. Ocupamos segmentos de entrenamiento, validación y prueba.

In [15]:
from datasets import DatasetDict

# Esta es la forma más fácil de partir los datos en tres que encontré

# Primero, se crea un split de entrenamiento del 30% del tamaño de todo
splitdata_all = data['train'].train_test_split(test_size=0.3, seed=7)

# Luego, se parte a la mitad de nuevo
# No hay una buena forma de cambiarle el nombre a los diccionarios así que
# el ['train'] de esta variable será el validation y el ['test'] será el test
splitdata_test_and_validation = splitdata_all['test'].train_test_split(test_size=0.5, seed=7)

# A partir de aquí, los nombres vuelven a la normalidad
data = DatasetDict({
    'train': splitdata_all['train'],
    'validation': splitdata_test_and_validation['train'],
    'test': splitdata_test_and_validation['test']
})

# Ahora, se borran las variables temporales para no quedarnos sin memoria RAM
del splitdata_all
del splitdata_test_and_validation

Ahora, tenemos varios `datasets` que podemos usar con distintos propósitos:

In [16]:
print(data['train'][1])
print(data['validation'][1])
print(data['test'][1])

{'text': "Hemiphora es un género de plantas con flores de la familia de las lamiáceas. En su aceptación actual ampliada, incluye 5 especies aceptadas; previamente, el género era considerado monoespecífico, con Hemiphora elderi como única especie. Es estrictamente endémico de Australia occidental (Estado de Western Australia).\n\nDescripción \nSon matas perennes de poca altura (unos 50cm), densamente peludo-lanosas con pelos dendroides, de tallo erecto ramificado con hojas simples, densamente peludas, enteras o serradas, sésiles y algo decurrentes, opuestas o decusadas o en verticilos de 3, pudiendo parecer estrechas por el borde fuertemente revoluto, de haces rugosas o con hinchazones. Las flores son axilares y solitarias, zigomórficas, hermafroditas, bibracteoladas, con el cáliz profundamnete pentalobulado y de tubo corto, mientras la corola es bilabiada, pentamera, con tubo largo, curvado y distalmente dilatado y con el labio superior bilobulado y el inferior trilobulado. Hay 4 estam

Nuestros `datasets` tiene un problema grave: la computadora no lo puede entender. Para solucionarlo, debemos **tokenizarlo**.

In [ ]:
from transformers import AutoTokenizer

# Estas variables especifican cómo vamos a entrenar a nuestro modelo
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

def tokenize_function(examples):
  return tokenizer(examples["text"])

# Esto recorre a través de todos nuestros datos y los tokeniza
tokenized_datasets = data.map(tokenize_function, batched=True, batch_size=128, num_proc=8, remove_columns=["text"])

Map (num_proc=8):   0%|          | 0/895038 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1584 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1104 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (15947 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1207 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3492 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence len

En el siguiente segmento de código se puede observar como los `datasets` ahora son un montón de números ilegibles.

In [11]:
print(tokenized_datasets['train'][1])
print(tokenized_datasets['validation'][1])
print(tokenized_datasets['test'][1])

{'input_ids': [34, 217, 337, 79, 13748, 434, 10517, 5256, 1614, 6697, 414, 830, 353, 437, 1767, 5109, 402, 5859, 12, 554, 5443, 1062, 1359, 427, 353, 2480, 161, 76, 5712, 12, 12342, 2036, 83, 18750, 311, 37, 1812, 65, 24307, 9, 14, 158, 158, 44, 10689, 529, 257, 73, 3481, 159, 158, 34, 217, 337, 79, 359, 1979, 85, 242, 314, 196, 502, 353, 3488, 2730, 4371, 807, 353, 338, 18910, 5256, 12, 5443, 1492, 271, 256, 230, 22717, 1941, 69, 1614, 6697, 414, 830, 407, 2480, 118, 78, 2310, 348, 5859, 353, 821, 82, 3655, 308, 224, 467, 66, 3415, 15333, 1044, 202, 280, 65, 14, 158, 158, 39, 69, 432, 314, 70, 5859, 159, 158, 37, 83, 434, 224, 467, 66, 3415, 353, 904, 8483, 24307, 9543, 336, 3895, 554, 434, 4102, 82, 3895, 2102, 234, 196, 250, 316, 224, 346, 1614, 904, 563, 12626, 229, 372, 79, 21442, 348, 388, 6869, 408, 69, 353, 5443, 267, 220, 19400, 5859, 14, 158, 158, 44, 316, 219, 316, 11685, 353, 5443, 1492, 271, 256, 407, 1603, 821, 720, 79, 2071, 18013, 316, 407, 2345, 213, 660, 1098, 18632, 

Luego, hay que procesarla y dividirla en bloques de cierta longitud. Es importante limitar la variable block_size para que nuestros datos entren dentro de la memoria de la `GPU` o `TPU` y también dentro del límite de **tokens** de la IA, que en este caso es de `tokenizer.model_max_length`, variable que se puede usar para adaptar el código automáticamente.

In [20]:
print(tokenizer.model_max_length)

1024


In [ ]:
block_size = tokenizer.model_max_length

# No entiendo nada de esta función así que no la podría explicar
# Solo sé que se supone que rompe a los datos en bloques de longitud block_size
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# Esto aplica la función group_texts(examples) a todos nuestros datos
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=128,
    num_proc=8,
)

Ahora, todos nuestros `datasets` están divididos en bloques de `tokenizer.model_max_length` y podemos entrenar a nuestro modelo.

In [21]:
print(lm_datasets['train'][1])
print(lm_datasets['validation'][1])
print(lm_datasets['test'][1])

{'input_ids': [24754, 349, 65, 15333, 540, 69, 353, 355, 441, 65, 8379, 3233, 353, 472, 626, 20353, 65, 407, 3545, 780, 278, 226, 14, 1781, 11218, 3796, 15333, 553, 196, 1045, 3655, 554, 1603, 22305, 9453, 353, 20353, 65, 5443, 250, 1517, 65, 196, 542, 85, 373, 217, 3545, 1370, 351, 257, 233, 216, 407, 196, 232, 257, 320, 69, 434, 194, 208, 85, 834, 69, 313, 1603, 586, 353, 21442, 5443, 196, 1045, 213, 14, 158, 158, 45, 668, 427, 928, 374, 427, 554, 1603, 230, 21499, 624, 228, 21442, 3470, 407, 13748, 434, 928, 374, 3444, 69, 313, 1301, 226, 231, 778, 512, 1808, 1346, 65, 691, 257, 233, 217, 399, 313, 250, 316, 1470, 22717, 503, 79, 230, 85, 89, 313, 375, 17952, 821, 202, 3545, 441, 65, 8379, 15753, 15333, 355, 13436, 69, 1137, 351, 11488, 554, 13451, 1083, 303, 1823, 20109, 226, 14, 5182, 11442, 1310, 325, 79, 21442, 3470, 227, 68, 336, 5859, 826, 6698, 1973, 17952, 1722, 830, 12, 20353, 65, 1375, 529, 65, 434, 65, 214, 3070, 65, 554, 355, 2934, 65, 1346, 65, 3933, 217, 1603, 24774, 1

# Entrenamiento

Ahora que nuestros datos estan listos, ya podemos crear a Wikibot-3000. Para eso vamos a usar un programa denomidado `trainer`, lo que significa entrenador. Primero, es necesario definir al modelo (o la plantilla, dicho en otras palabras) que vamos a usar.

In [22]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

Solo falta definir unos cuantos datos y después se podrá entrenar al modelo.

In [27]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    ".Wikibot-3000",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

ImportError: ignored

Para entrenar el modelo le pasamos todos nuestros datos a la clase `Trainer`. Es como si un entrenador le estuviese explicando a nuestra máquina a escribir desde cero, enseñándole cómo se utiliza un teclado, que es un verbo, un sustantivo, etc.

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['validation'],
)

trainer.train()

Ahora podemos evaluar a nuestro modelo para evaluar su rendimiento.

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Puntaje: {math.exp(eval_results['eval_loss']):.2f}")

Lo único que falta es subir el modelo a [Hugging Face](https://huggingface.co) para compartirlo y ejecutarlo con facilidad.

In [ ]:
trainer.push_to_hub()

# Bibliografía

https://colab.research.google.com/notebooks/tpu.ipynb
https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/language_modeling_from_scratch.ipynb
https://huggingface.co/datasets/graelo/wikipedia/viewer/20230601.es
https://huggingface.co/docs/datasets/v2.14.5/en/index